In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.cm as cm

# === Parameters ===
N = 50
tolerance = 1e-6
max_iter = 10000
omegas = np.round(np.arange(1.80, 2.00, 0.01), 3)  # 0.01步长

# === Initialize Grid Function (保持一致) ===
def initialize_grid(N):
    c = np.zeros((N+1, N+1))
    c[:, -1] = 1.  # 右边界
    return c

# === Update Boundary Conditions (Periodic) ===
def update_boundary(c):
    c[0, :] = c[-2, :]
    c[-1, :] = c[1, :]
    c[:, 0] = 0.
    c[:, -1] = 1.
    return c

# === SOR Method with Objects ===
def SOR_sink_obj(c0, omega, objects, tolerance=tolerance, max_iter=max_iter):
    assert objects.shape == c0.shape, "The objects must have the same shape as the concentration grid."
    N = c0.shape[0]
    c = c0.copy()
    c[objects] = 0.
    deltas = []
    for _ in range(max_iter):
        # Boundary conditions
        c[:, -1] = 1.
        c[:, 0] = 0.
        c[0, :] = c[-2, :]
        c[-1, :] = c[1, :]
        # Reset delta
        delta = 0.
        for i in range(1, N-1):
            for j in range(1, N-1):
                if objects[i, j]:  # If this point is an object, skip iteration
                    c[i, j] = 0
                    continue
                old_c = c[i, j]
                c[i, j] = omega * 0.25 * (c[i-1, j] + c[i+1, j] + c[i, j-1] + c[i, j+1]) + (1 - omega) * c[i, j]
                delta = max(delta, abs(c[i, j] - old_c))
        deltas.append(delta)
        if delta <= tolerance:
            break
    return c, np.array(deltas)

# === Object Configurations ===
c0 = initialize_grid(N)

# No Object (0)
objects0 = np.zeros((N+1, N+1), dtype=bool)  # No object

# One Square
objects1 = np.zeros((N+1, N+1), dtype=bool)
objects1[2*N//5:3*N//5, 2*N//5:3*N//5] = True

# Two Squares
objects2 = np.zeros((N+1, N+1), dtype=bool)
objects2[N//5:2*N//5, 2*N//5:3*N//5] = True
objects2[3*N//5:4*N//5, 2*N//5:3*N//5] = True

# Three Squares
objects3 = np.zeros((N+1, N+1), dtype=bool)
objects3[2*N//5:3*N//5, 3*N//5:4*N//5] = True
objects3[N//5:2*N//5, N//5:2*N//5] = True
objects3[3*N//5:4*N//5, N//5:2*N//5] = True

# Four Squares
objects4 = np.zeros((N+1, N+1), dtype=bool)
objects4[N//5:2*N//5, N//5:2*N//5] = True
objects4[3*N//5:4*N//5, N//5:2*N//5] = True
objects4[N//5:2*N//5, 3*N//5:4*N//5] = True
objects4[3*N//5:4*N//5, 3*N//5:4*N//5] = True

# === Experimenting with Different Object Configurations ===
obj_configs = [objects0, objects1, objects2, objects3, objects4]
optimal_omegas = []
iterations = []

for obj in obj_configs:
    obj_iters = []
    converged = np.ones_like(omegas).astype(bool)
    for omega in omegas:
        try:
            _, deltas_sor = SOR_sink_obj(c0, omega=omega, objects=obj)
        except Exception as e:
            converged[omegas == omega] = 0
        else:
            if len(deltas_sor) == max_iter:
                converged[omegas == omega] = 0
            else:
                obj_iters.append(len(deltas_sor))
    iterations.append(obj_iters)
    optimal_omegas.append(omegas[converged][np.argmin(obj_iters)])

print("Optimal Omegas:", optimal_omegas)

# === Plot Results (保持原答案风格，但样式改进) ===
obj_nums = [0, 1, 2, 3, 4]
fig, ax1 = plt.subplots(figsize=(8, 6))
ax1.set_xlabel(r'Number of $\frac{N}{5}\times\frac{N}{5}$ Objects')
ax1.plot(obj_nums, optimal_omegas, 'o-', label='Optimal ω', color='blue')
ax1.set_ylabel('Optimal ω', color='blue')
ax2 = ax1.twinx()
ax2.plot(obj_nums, [min(iters) for iters in iterations], 'o-', label='Iterations', color='red')
ax2.set_ylabel('Iterations ($\omega = 1.89$)', color='red')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.grid(True, linestyle='--', alpha=0.5)
plt.title('Optimal ω and Iterations vs Number of Objects')
plt.show()
